In [1]:
import pandas as pd
from nltk.corpus import stopwords
import enchant
import re

In [2]:
X = pd.read_csv('DATA/input_train.csv', sep=";")
y = pd.read_csv('DATA/output_train.csv', sep=";")

features = X.columns
targets = y['intention'].unique

In [3]:
features

Index(['﻿ID', 'question'], dtype='object')

In [4]:
#import the list of medicaments
text_file = open("DATA/medicaments.txt", "r")
medicaments = [x[:-1] for x in text_file.readlines()]
posologies = ["mg", "g", "ml", "l"]
stop_words = stopwords.words('french')
correcteur = enchant.Dict("fr_FR")

In [5]:
# Indicateur d'avancement 
total = len(X['question'])

In [7]:
def clean(word):
    word = word.lower()
    if word in medicaments:
        word = "NOM_DE_MEDICAMENT"
    elif word[-2:] in posologies:
        word = "POSOLOGIE"
    elif not correcteur.check(word):
        suggestions = correcteur.suggest(word)
        if type(suggestions) == 'list':
            distances = [enchant.utils.levenshtein(s, word) for s in suggestions]
            min_index = distances.index(min(distances))
            word = suggestions[min_index]
        elif type(suggestions) == 'str':
            word = suggestion
        
    return word

def preprocess(row):
    sentence = re.sub(r'[^\w]', ' ', row['question'])
    word_list = [clean(word) for word in sentence.split() if word not in stop_words]
    preprocessed_sentence = ' '.join(word for word in word_list)
    if row['﻿ID']%100 == 0:
        print(" row {} / {}".format(row['﻿ID'], total))
    return preprocessed_sentence

def output_format(row):
    return row[0]

In [14]:
X_clean = X.apply(preprocess, axis = 1)

 row 0 / 8028
 row 100 / 8028
 row 200 / 8028
 row 300 / 8028
 row 400 / 8028
 row 500 / 8028
 row 600 / 8028
 row 700 / 8028
 row 800 / 8028
 row 900 / 8028
 row 1000 / 8028
 row 1100 / 8028
 row 1200 / 8028
 row 1300 / 8028
 row 1400 / 8028
 row 1500 / 8028
 row 1600 / 8028
 row 1700 / 8028
 row 1800 / 8028
 row 1900 / 8028
 row 2000 / 8028
 row 2100 / 8028
 row 2200 / 8028
 row 2300 / 8028
 row 2400 / 8028
 row 2500 / 8028
 row 2600 / 8028
 row 2700 / 8028
 row 2800 / 8028
 row 2900 / 8028
 row 3000 / 8028
 row 3100 / 8028
 row 3200 / 8028
 row 3300 / 8028
 row 3400 / 8028
 row 3500 / 8028
 row 3600 / 8028
 row 3700 / 8028
 row 3800 / 8028
 row 3900 / 8028
 row 4000 / 8028
 row 4100 / 8028
 row 4200 / 8028
 row 4300 / 8028
 row 4400 / 8028
 row 4500 / 8028
 row 4600 / 8028
 row 4700 / 8028
 row 4800 / 8028
 row 4900 / 8028
 row 5000 / 8028
 row 5100 / 8028
 row 5200 / 8028
 row 5300 / 8028
 row 5400 / 8028
 row 5500 / 8028
 row 5600 / 8028
 row 5700 / 8028
 row 5800 / 8028
 row 5900

In [19]:
X_clean.head()

0    bonjour trompé forum question alors repose ici...
1       NOM_DE_MEDICAMENT soulagera contre les nausées
2    médecin NOM_DE_MEDICAMENT prescrit NOM_DE_MEDI...
3    est existe forme adaptée enfant 5ans NOM_DE_ME...
4    medecin soigne NOM_DE_MEDICAMENT pharingite NO...
dtype: object

In [17]:
X_clean.to_csv('DATA/clean_input_train.csv', sep=';', header = ['question'])